In [62]:
#importing all the required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.backends.backend_pdf as pdf
import os
import sys
import csv
import pandas as pd
import itertools
import random
import time
#file paths for directories relative to current directory
root_dir = os.path.join(os.pardir,os.pardir)
path = ''
try:
    os.mkdir(os.path.join(root_dir,'Output'))
except: #in case any error occurs pass
    pass
try:
    os.mkdir(output_dir)
except:
    pass

In [63]:
def get_data(data):
    lines = data.readlines()            #forming a list of lines
    #empty lists/dictionaries to store the parsed information
    metadata={}                         
    voter=[]                            
    projects=[]                         
    for line in lines:
        #using the rstrip fucntion to strip space by default 
        line = line.rstrip()        
        if(line == 'META'):
            state = 'meta'
        elif(line == 'PROJECTS'):
            state = 'project'
        elif(line == 'VOTES'):
            state = 'vote'
        else:
            #spliting the line separated by ; into list of strings on satisfying a given condition
            if(state == 'meta'):
                line = line.split(';')
                metadata[line[0]] = line[1]
            elif(state == 'project'):
                line = line.split(';')
                projects.append(line)
            elif(state == 'vote'):
                line = line.split(';')
                voter.append(line)
    #projects_19.pop(0)
    #removing key element from the metadata
    metadata.pop('key')
    #returning the three lists/dictionary
    return metadata, projects, voter

In [64]:
def intersecter(projects_1,projects_2):
    #iterating through each element in projects_1
    for i in projects_1:
        #checking if the element at 4th index of i is present in the values of index 4 of all elements of projects_2
        if i[4] in projects_2[:,4]:
            print(i)

In [65]:
def vote_vector_create(votes,num_projects,ids):
    #create a numpy array of zeros with dimensions len(votes) and num_projects-1+len(votes[0]) and of type uint8 and object
    vote_vector = np.zeros((len(votes), num_projects-1+len(votes[0]))).astype(np.uint8).astype(object)
    #copying the first 4 elements of the first vote into the first row of the vote vector
    vote_vector[0,:4] = votes[0][:4]
    #copying the 6th element of the first vote into the 5th element of the first row of the vote vector
    #vote_vector[0,1] = votes[0][2]
    #iterating through the indices of the ids list
    for i in range(len(ids)):
        #adding a string to the ith element of the first row of the vote vector
        vote_vector[0,4+i] = 'approval on project ' +ids[i]
    #iterating through the indices of the votes list starting at 1
    for i in range(1,len(votes)):
        #copying the first 4 elements of the ith vote into the ith row of the vote vector
        vote_vector[i,0] = votes[i][0]
        vote_vector[i,1] = votes[i][1]
        vote_vector[i,2] = votes[i][2]
        vote_vector[i,3] = votes[i][3]
        #copying the 6th element of the ith vote into the 5th element of the ith row of the vote vector
        ##vote_vector[i,4] = votes[i][5]
        #spliting the 5th element of the ith vote into a list of approved projects
        list_of_approved_projects = votes[i][4].split(',')
        #iterating through the list of approved projects 
        for project in list_of_approved_projects:
            #finding the index of the project in the ids list and set the corresponding element in the ith row of the vote vector to 1
            vote_vector[i-1,4+ids.index(project)] = 1
    #returning the vote vector
    return vote_vector


In [66]:
def preprocess(file):
    #opening the file in the input directory and joining the directory and file name using os.path.join
    data_19 = open(file,'r') 
    #calling get data function and passing data_19 as argument and storing the returned values in metadata_19, projects_19 and votes_19
    metadata_19, projects_19, votes_19 = get_data(data_19)
    #converting data into a numpy array
    projects_19 = np.array(projects_19)
    #creating a dataframe using projects_19 array
    dataframe_19 = pd.DataFrame(projects_19)
    #writing the contents of dataframe into csv file using to_csv(), the columns of the csv file will not have headers
    dataframe_19.to_csv(os.path.join(path,'projects_19.csv'),header=False)
    #caling vote_vector_create function and extracting project ids from projects_19
    vote_vector = vote_vector_create(votes_19,len(projects_19)-1,list(projects_19[1:,0]))
    #creating a dataframe using vote_vector data
    vote_dataframe = pd.DataFrame(vote_vector)
    #writing the contents of dataframe into csv file using to_csv(), the columns of the csv file will not have headers
    vote_dataframe.to_csv(os.path.join(path,"votes.csv"),header=False)

In [67]:
def find_target_cost(string):
    #assigning a numerical "cost" to the input string based on whether certain keywords are present in the string
    #initializing cost to 0
    cost = 0.0
    if '3rd' in string:
        cost = cost + 0.211
    if '4th' in string:
        cost = cost + 0.053
    if 'lect' in string:
        cost = cost + 0.224
    if '1st' in string:
        cost = cost + 0.145
    if '2nd' in string:
        cost = cost + 0.263
    if 'ann' in string:
        cost = cost + 0.000
    if 'total' in string:
        cost = cost + 0.092
    if 'pd' in string:
        cost = cost + 0.013
    
    return cost

In [68]:
def find_normalized_cost(val, mu, sigma):
    #returning the normalized cost
    #mu -> mean of distribution, sigma -> standard deviation 
    return (val-mu)/sigma

In [69]:
def find_categorical_cost(string):
    #assigning a numerical "cost" to the input string based on whether certain keywords are present in the string
    #initializing cost to 0
    cost = 0.0
    if 'culture' in string:
        cost = cost + 0.136
    if 'education' in string:
        cost = cost + 0.227
    if 'sport' in string:
        cost = cost + 0.212
    if 'sport1' in string:
        cost = cost + 0.091
    if 'sport2' in string:
        cost = cost + 0.030
    if 'sport3' in string:
        cost = cost + 0.106
    if 'gym' in string:
        cost = cost + 0.030
    if 'medi' in string:
        cost = cost + 0.167
    if 'women' in string:
        cost = cost + 0.000
        
    return cost

In [70]:
def reward_calculator(cat_frac, tar_frac, time_cons, budget, selectivity, popularity):
##def reward_calculator(demo_frac, time_cons):
    pos_reward = (1*((cat_frac+tar_frac)/2))*selectivity
    #mapping the time spent on the website (in hours) to a value between -1 and 1
    # neg_reward = 2*np.arctan(time_cons)/np.pi
    neg_reward = time_cons/budget
    # pos_reward = ((pos_reward))*2
    reward = pos_reward - neg_reward
    # reward = ((reward + 1)/2)*4+1
    return reward

In [71]:
def max_find_list(dictionary):
    #finding the maximum value in a dictionary, considering that some values may be lists, 
    #then returning the maximum value and a list of keys that have that value
    #initializing max to be negative infinity
    Max = -np.inf
    for value in dictionary.values():
        if(type(value)==list):
            Max = max(Max, max(value))
        else:
            Max = max(Max, value)
    key_list=[]
    for key, value in dictionary.items():
        if(type(value)==list and Max == max(value)):
            key_list.append(key)
        elif(Max == value):
            key_list.append(key)
    return Max, key_list
def max_find(dictionary):
    #finding the maximum value in a dictionary, assuming that all values are integers, 
    #then returning the maximum value and a single key that has that value.
    Max = -np.inf
    for value in dictionary.values():
        Max = max(Max, value)
    key_list=[]
    for key, value in dictionary.items():
        if(Max == value):
            key_list = key
    return Max, key_list
def min_find(dictionary):
    #finding the minimum value in a dictionary, assuming that all values are integers, 
    #then returning the minimum value and a single key that has that value.
    Min = np.inf
    for value in dictionary.values():
        Min = min(Min, value)
    key_list=[]
    for key, value in dictionary.items():
        if(Min == value):
            key_list = key
    return Min, key_list
def random_find(dictionary):
    #selecting a random key-value pair from the dictionary and returning the value and the key
    dictionary = {k: v for k,v in dictionary.items() if v}
    key, value = random.choice(list(dictionary.items()))
    return value, key

In [72]:
#categories is a list of category names
#list_is is a list of strings where each string represents a set of categories separated by commas
def category_combination(categories, list_is):
    hash = {} #empty dictionary
    for i in range(len(list_is)):
        #converting each string in list_is to a set of categories using the split() function
        list_is[i] = set(list_is[i].split(','))
    for i in range(len(categories),0,-1):
        #generating all possible combinations of categories
        combinations = itertools.combinations(categories,i)
        for combination in combinations:
            combination = [x for x in combination]
            #checking if the set of categories exists in list_is, setting corresponding value to 1 if it does otherwise 0
            hash[','.join(combination)] = 1 if set(combination) in list_is else 0
            #print(hash[','.join(combination)])
    #reateing a Pandas DataFrame from the hash dictionary, with the keys as the first column and the values as the second column.
    df = pd.DataFrame(list(hash.items()),columns=['combination','indicator'])
    #writing the contents of dataframe into csv file using to_csv(), the columns of the csv file will not have headers
    df.to_csv(os.path.join(path,'combination_category.csv'))
def target_combination(categories, list_is):
    #This function is similar to the previous function, but it skips the combinations that include 'None'
    hash = {}
    for i in range(len(list_is)):
        list_is[i] = set(list_is[i].split(','))
    for i in range(len(categories),0,-1):
        combinations = itertools.combinations(categories,i)
        for combination in combinations:
            if 'None' in combination and i > 1:
                pass
            else:
                combination = [x for x in combination]
                hash[','.join(combination)] = 1 if set(combination) in list_is else 0
                #print(hash[','.join(combination)])
    df = pd.DataFrame(list(hash.items()),columns=['combination','indicator'])
    df.to_csv(os.path.join(path,'combination_target.csv'))       

In [73]:
#Parameter list:
# - voter_list: a DataFrame containing voter information
# - learning_rate: a float that determines how much weight to give new information
# - projects: a DataFrame containing information about available projects
# - communication_graph: a list of lists representing connections between voters
# - budget: a float representing the maximum budget for a combination of projects
# - output_file: a file object to write output to (default is standard output)
# - num_iter: an integer specifying the number of iterations to run the algorithm for
# - halfselector: an integer that determines how combinations are selected
# - epsilon: a float that determines the probability of choosing a random combination
# - decay: a float that determines how much weight to give previous information
# - is_complete_graph: a boolean that determines if the communication graph is fully connected
# - communicated: a function that determines how information is communicated between voters
# - randomised_communication: a boolean that determines if the communication graph is randomised
# - max_degree: an integer that determines the maximum degree of the communication graph
# - Q_interval: an integer that determines how often to print the Q table
# - agent_needed: an integer that determines how many agents are needed for a project
def Q_learn(valid_combinations, voter_list, learning_rate, projects, communication_graph, budget, output_file=sys.stdout, num_iter=100, halfselector = 0, epsilon=0.001, decay = 0.2, is_complete_graph = 0, communicated = min_find, randomised_communication = 0, max_degree = 2, Q_interval = 100, agent_needed = 1):
    #projects.sort_values(by='project_id')
    # projects['demographics_fraction'] = [find_demo_fraction(x) for x in projects['target']]
    #Preprocess project data to calculate normalized cost, reward and time consumed
    projects['category_fraction'] = [find_categorical_cost(x) for x in projects['category']]
    projects['target_fraction'] = [find_target_cost(x) for x in projects['target']]
    #Calculating the mean and standard deviation of project costs
    mu = projects['cost'].mean()
    sigma = projects['cost'].std()
    #Adding columns to the projects DataFrame to store normalized costs and rewards
    projects['norm_cost'] = [find_normalized_cost(x, mu, sigma) for x in projects['cost']]
    ##projects['reward'] = [reward_calculator(a,b,c,d) for a, b, c, d in zip(projects['demographics_fraction'], projects['cat_fraction'], projects['target_fraction'], projects['cost'])]
    projects['reward'] = [reward_calculator(b,c,d,budget,e,a) for b, c, d, e, a in zip(projects['category_fraction'], projects['target_fraction'], projects['cost'], projects['selection fraction'], projects['popularity percentage'])]
    print(projects[['project id','reward']],file=output_file)
    #Creating a dictionary to store the Q-table for each voter
    voter_Q_table = {}
    #Creating a dictionary to store the initial Q-values for each combination of projects
    Initial_Q_table = {}
    # Generating a list of all possible combinations of projects
    list_of_combinations = []
    # num = 0
    count=0
    for i in range(len(projects['project id']),0,-1):
        combinations = itertools.combinations(projects['project id'], i)
        for combination in combinations:
            data_frame = projects.loc[projects['project id'].isin(combination)]
            sum = np.sum(data_frame['cost'].to_list())
            num=random.randint(0,12)
            # print(sum)
            # Only adding combinations with cost under the budget constraint
            if(sum < budget and count<valid_combinations):
            # if(sum < budget and num<10):
            # if(sum < budget and count<12):
            # if(sum < budget):
                flag = 0
                for lst in list_of_combinations:
                    if(all(x in lst for x in combination)):
                        # print('',lst,'')
                        flag = 1
                        break
                if(not flag):
                    count = count + 1
                    list_of_combinations.append(combination)
                    Initial_Q_table[combination] = 0
                    
    # list_of_combinations = []
    # count = len(list_of_combinations)
    #Initializing the Q-table for each combination of projects
    for combination in list_of_combinations:
        Initial_Q_table[combination] = 0
    #Getting the list of voter IDs
    voter_id_list = voter_list['0'].to_list()
    # print(voter_list)
    print("Valid Combinations are:",file=output_file)
    for combination in list_of_combinations:
        print(combination,file=output_file)
    #Initializing each voter's choice randomly
    initial_choice = np.zeros(len(voter_id_list),dtype=object)
    for i in range(len(voter_id_list)):
        max=0
        for combination in list_of_combinations:
            count1 = 0
            for index in range(0, len(combination), 1):
                project_number = int(combination[index])
                if str(voter_list.at[i, str(project_number)])=='1.0':
                    count1 += 1
            if count1>max:
                max=count1
                initial_choice[i] = combination    
            if max==0:
                # print(i)
                initial_choice[i] = list_of_combinations[np.random.randint(len(list_of_combinations))]          
        # initial_choice[i] = list_of_combinations[np.random.randint(len(list_of_combinations))]
        voter_Q_table[voter_id_list[i]] = Initial_Q_table.copy()
        current_table = voter_Q_table[voter_id_list[i]]
        # print(current_table)
        # print(initial_choice[i])
    # print(initial_choice)
    current_choice = initial_choice
    #Printing the initial choices
    print("Initial Choices:",file=output_file)
    print("---------------------------------------------------------------------------",file=output_file)
    for i in range(len(initial_choice)):
        print("Voter",str(i+1)+": ID",str(voter_id_list[i])+"; Choice",initial_choice[i],'',file=output_file)
    print("---------------------------------------------------------------------------",file=output_file)
    #Running the Q-learning algorithm for num_iter iterations
    for iteration in range(num_iter):
        #Creating an empty list to store the communicated projects for each voter
        #Randomly choosing a next choice with probability epsilon or choosing the best known choice otherwise
        communicated_projects = []
        #Iterating over each voter
        for i in range(len(voter_id_list)):
            a = np.random.random()
            if(a < epsilon or iteration == 0 or iteration == 1 or iteration == 2):
                #Choosing a random combination from the list of all possible combinations
                next_choice = list_of_combinations[np.random.randint(len(list_of_combinations))]
                # print(next_choice,1)
            else:
                #Choosing the best known choice from the Q-table for the current voter
                next_viable_choices = max_find_list(voter_Q_table[voter_id_list[i]])[1]
                next_choice = next_viable_choices[np.random.randint(len(next_viable_choices))]
                # print(next_choice,2)
            #Getting the current Q-table for the voter
            current_table = voter_Q_table[voter_id_list[i]]
            #Empty list to calculate the rewards for the current choice for the voter
            reward = []
            for choice in current_choice[i]:
                series = projects[projects['project id'] == choice]
                # print(float(series['reward']))
                reward.append(float(series['reward']))
            #Updating the Q-table for the voter based on the chosen next choice and the rewards received
            voter_Q_table[voter_id_list[i]][current_choice[i]] += learning_rate*(decay*current_table[next_choice]-current_table[current_choice[i]]+np.sum(np.array(reward)))
            #Setting the current choice for the voter to the chosen next choice
            current_choice[i] = next_choice
            # print(min_find(current_table))
            #Adding the communicated projects for the voter to the list
            communicated_projects.append(communicated(voter_Q_table[voter_id_list[i]]))    # if minimum data is communicated, use min_find(cuurent_table)[1]. If maximum data is communicated, use max_find(current_table)[1][0]
        # print(communicated_projects)
        #Updating the Q-table for each voter based on the communicated projects
        for i in range(len(voter_id_list)):
            degree=[0]*(len(voter_list))
            if(is_complete_graph):
                relevant_communicated_projects = communicated_projects
            else:
                if(randomised_communication):
                    communication_graph=[]
                    # j = np.random.randint(max_degree-1,max_degree)
                    K = [x for x in range(len(voter_list)) if [x,i] not in communication_graph and [i,x] not in communication_graph]
                    P = random.sample(K,max_degree-degree[i])
                    for p in P:
                        if(degree[p]<max_degree):
                            communication_graph.append([p,i])
                            degree[i]+=1
                adjacent_points = []
                for edge in communication_graph:
                    if(edge[1] == i):
                        adjacent_points.append(edge[0])
                    if(edge[0] == i):
                        adjacent_points.append(edge[1])
                current_table = voter_Q_table[voter_id_list[i]]
                relevant_communicated_projects = [communicated_projects[i] for i in adjacent_points]
                # print(adjacent_points, file=output_file)
                # print(degree, file=output_file)
                # print(communication_graph, file=output_file)
            for reward, combination in relevant_communicated_projects:
                current_table[combination] = reward
        # print(degree, file=output_file)
        #Printing the current choices for each voter after the iteration
        print("After Iteration",iteration+1,":",file=output_file)
        for i in range(len(current_choice)):
            print("Voter",str(i+1)+": ID",str(voter_id_list[i])+"; Choice",current_choice[i],'',file=output_file)
        print("---------------------------------------------------------------------------",file=output_file)
        # count = valid_combinations
        #Checking if consensus has been reached (i.e., all voters have chosen the same combination)
        if(len(set(current_choice))==1):
            print("Concensus reached after",iteration+1,"iterations.",file=output_file)
            print("Max degree:",max_degree,"Number of valid combinations:",count,"Using random find.",file=output_file)
            break
        if(iteration % Q_interval == 0):
            print("---------------------------------------------------------------------------",file=output_file)
            print(voter_Q_table[voter_id_list[agent_needed]],file=output_file)
    return voter_Q_table, current_choice

In [74]:
#Run Once
preprocess('Student_elections_input.pb')

In [ ]:
for count in range(13,14,1):
    for maxdegree in range(4, 21, 6):
        projects_19 = pd.read_csv(os.path.join(path,'Student_elections_projects.csv'))
        # select projects from the csv manually
        sample_projects = projects_19.loc[projects_19['project id'].isin([1142,2296,2263,1867,2037,518,248,1226,2205])]
        voter = pd.read_csv(os.path.join(path,'votes_p.csv'))
        # select how many voters you want
        # sample_voter = voter.sample(n=50)
        sample_voter = voter.head(n=50)
        # sample_voter = voter.loc[voter['voter_id'].isin([333, 898, 1307,2155,2558,3054,3232,3523,3874,3892,4099,4576,4625,4713,4770,4875,5139,5428,5445,5735,6119,6123,6243,6272,6739,6746,6755,6994,7005,7790,7818,7843,8645,9230,9314,9444,9594,9662,10043,10145,10209,10211,10718,10750,10948,11074,11173,11322,11328,11697])]
        sample_voter.reset_index(drop=True, inplace=True)
        edge_list = []
        OutputFile = open(os.path.join(path,'Output/P_Max'+str(maxdegree)+'_comb'+str(count)),'w')
        Q=Q_learn(count,sample_voter,0.1,sample_projects,np.array(edge_list),550000, output_file=OutputFile, num_iter=10000,is_complete_graph=0, communicated=random_find,epsilon=0.01, randomised_communication=1,max_degree=maxdegree)[1]
        print('There are',len(set(Q)),'opinions after all iterations.',file=OutputFile)
        for L in set(Q):
            print(L,'count:',list(Q).count(L),file=OutputFile)
        pass
        OutputFile.close()